In [ ]:
import gym
import numpy as np
from numpy import random
import time
env=gym.make('MountainCar-v0')
import matplotlib.pyplot as plt 

In [ ]:
DESCRETE_OS_SIZE=[20]*2
descrete_os_win=(env.observation_space.high-env.observation_space.low)/DESCRETE_OS_SIZE
q_table=np.random.uniform(low=-2,high=0,size=DESCRETE_OS_SIZE+[env.action_space.n])
np.shape(q_table)
print(q_table)
def get_descrete_state(state):
    descrete_state=(state-env.observation_space.low)/(descrete_os_win)
    return  tuple(descrete_state.astype(np.int32))

In [ ]:
ep_rewards=[]
stats={'ep':[],'avg':[],'min':[],'max':[]}
SHOW_EVERY=500
epi_rewards=[]

In [ ]:
alpha=0.1
gamma=0.95
EPISODES=2000
episilon=0.5
start_decay=1
end_decay=EPISODES//2
decay_value=episilon/(end_decay-start_decay)
for episode in range(EPISODES):
    ep_reward=0
    descrete_state=get_descrete_state(env.reset())
    if episode%2000 ==0:
        render=True
        print(episode)
    else:
        render=False
    done=False
    while not done:
   
        if np.random.random()>episilon:
            action=np.argmax(q_table[descrete_state[0]][descrete_state[1]])
        else:
            action=env.action_space.sample()
        new_state,reward,done,_=env.step(action)
        ep_reward+=reward
        
        descrete_new_state=get_descrete_state(new_state)
        if render:
            # env.render()
            pass
        if not done:
            max_future_q=np.max(q_table[descrete_new_state])
            current_q=q_table[descrete_state][action]
            new_q=current_q+alpha*(reward+gamma*max_future_q-current_q)
            q_table[descrete_state][action]=new_q
        elif new_state[0]>=env.goal_position:
            q_table[descrete_state][action]=0
            # if(not render):
            # env.render()
            print(episode)
    epi_rewards.append(ep_reward)
    if episode%SHOW_EVERY==0:
        stats['avg'].append(sum(epi_rewards[-SHOW_EVERY:])/SHOW_EVERY)
        stats['ep'].append(episode)
        stats['max'].append(max(epi_rewards[-SHOW_EVERY:]))
        stats['min'].append(min(epi_rewards[-SHOW_EVERY:]))
    descrete_state=descrete_new_state
    if episode>=start_decay and episode<=end_decay:
        episilon-=decay_value
    if episode % 10 == 0:
        np.save(f"{episode}", q_table)
            # C:\Users\User\Desktop\Python
env.close()


    
     

In [ ]:
plt.plot(stats['ep'], stats['avg'], label="average rewards")
plt.plot(stats['ep'], stats['max'], label="max rewards")
plt.plot(stats['ep'], stats['min'], label="min rewards")
plt.legend(loc=1)
plt.show()

In [ ]:
from matplotlib import style
style.use('ggplot')


def get_q_color(value, vals):
    if value == max(vals):
        return "green", 1.0
    else:
        return "red", 0.3


fig = plt.figure(figsize=(12, 9))


for i in range(0,20000, 10):
    print(i)
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(313)

    q_table = np.load(f"{i}.npy")

    for x, x_vals in enumerate(q_table):
        for y, y_vals in enumerate(x_vals):
            ax1.scatter(x, y, c=get_q_color(y_vals[0], y_vals)[0], marker="o", alpha=get_q_color(y_vals[0], y_vals)[1])
            ax2.scatter(x, y, c=get_q_color(y_vals[1], y_vals)[0], marker="o", alpha=get_q_color(y_vals[1], y_vals)[1])
            ax3.scatter(x, y, c=get_q_color(y_vals[2], y_vals)[0], marker="o", alpha=get_q_color(y_vals[2], y_vals)[1])

            ax1.set_ylabel("Action 0")
            ax2.set_ylabel("Action 1")
            ax3.set_ylabel("Action 2")

    #plt.show()
    plt.savefig(f"{i}.png")
    plt.clf()

In [ ]:
import os
for i in range(0,6000,10):
    os.remove(f"{i}.npy")